In [1]:
from ptrail.core.Datasets import Datasets
from ptrail.core.TrajectoryDF import PTRAILDataFrame
from ptrail.utilities.conversions import Conversions
from ptrail.regularization.compression import Compression
import pandas as pd

In [2]:
dataset = PTRAILDataFrame(data_set=pd.read_csv('./data/starkey.csv'),
                          datetime='DateTime', traj_id='traj_id', latitude='lat', longitude='lon')
dataset

lat         lon  StarkeyTime    GMDate  \
traj_id   DateTime                                                            
880109D01 1995-04-13 13:40:06  45.239682 -118.533204    229902006  21:40:06   
          1995-04-15 12:16:15  45.250521 -118.530438    230069775  20:16:15   
          1995-04-15 21:39:38  45.247943 -118.541455    230103578  05:39:38   
          1995-04-16 03:32:14  45.247429 -118.539530    230124734  11:32:14   
          1995-04-16 04:08:28  45.247117 -118.542579    230126908  12:08:28   
...                                  ...         ...          ...       ...   
OSUX93191 1996-08-15 06:51:06  45.220642 -118.543392    272213466  14:51:06   
          1996-08-15 08:45:15  45.219785 -118.546807    272220315  16:45:15   
          1996-08-15 10:36:54  45.219801 -118.545661    272227014  18:36:54   
          1996-08-15 12:31:22  45.220268 -118.551024    272233882  20:31:22   
          1996-08-15 14:25:58  45.222562 -118.541151    272240758  22:25:58   

                                 GMTime   LocDate   LocTime  RadNum  Species  \
traj_id   DateTime                                                             
880109D01 1995-04-13 13:40:06  19950413  19950413  13:40:06     409        0   
          1995-04-15 12:16:15  19950415  19950415  12:16:15     409        0   
          1995-04-15 21:39:38  19950416  19950415  21:39:38     409        0   
          1995-04-16 03:32:14  19950416  19950416  03:32:14     409        0   
          1995-04-16 04:08:28  19950416  19950416  04:08:28     409        0   
...                                 ...       ...       ...     ...      ...   
OSUX93191 1996-08-15 06:51:06  19960815  19960815  06:51:06     390        2   
          1996-08-15 08:45:15  19960815  19960815  08:45:15     390        2   
          1996-08-15 10:36:54  19960815  19960815  10:36:54     390        2   
          1996-08-15 12:31:22  19960815  19960815  12:31:22     390        2   
          1996-08-15 14:25:58  19960815  19960815  14:25:58     390        2   

                                 UTME     UTMN  Year  Grensunr  Grensuns  \
traj_id   DateTime                                                         
880109D01 1995-04-13 13:40:06  379662  5010734    95  13:13:00  02:39:00   
          1995-04-15 12:16:15  379895  5011927    95  13:09:00  02:41:00   
          1995-04-15 21:39:38  379039  5011656    95  13:07:00  02:43:00   
          1995-04-16 03:32:14  379188  5011581    95  13:07:00  02:43:00   
          1995-04-16 04:08:28  378938  5011567    95  13:07:00  02:43:00   
...                               ...      ...   ...       ...       ...   
OSUX93191 1996-08-15 06:51:06  378821  5008634    96  12:56:00  03:04:00   
          1996-08-15 08:45:15  378568  5008518    96  12:56:00  03:04:00   
          1996-08-15 10:36:54  378645  5008543    96  12:56:00  03:04:00   
          1996-08-15 12:31:22  378232  5008600    96  12:56:00  03:04:00   
          1996-08-15 14:25:58  378995  5008820    96  12:56:00  03:04:00   

                               Obswt  
traj_id   DateTime                    
880109D01 1995-04-13 13:40:06   1.47  
          1995-04-15 12:16:15   1.59  
          1995-04-15 21:39:38   1.34  
          1995-04-16 03:32:14   1.50  
          1995-04-16 04:08:28   1.34  
...                              ...  
OSUX93191 1996-08-15 06:51:06   1.60  
          1996-08-15 08:45:15   1.39  
          1996-08-15 10:36:54   1.43  
          1996-08-15 12:31:22   1.53  
          1996-08-15 14:25:58   1.66  

[287136 rows x 15 columns]

In [3]:
# adapt from https://github.com/uestc-db/traj-compression/blob/master/batch/TD-TR/TD-TR.cpp

import numpy as np
import time

import pandas as pd


def calc_SED(pA, pI, pB):
    """
    It computes the Synchronous Euclidean Distance (SED) error

    :param pA: initial point
    :param pI: middle point
    :param pB: final point
    :return: SED error
    """
    pA_lat, pA_lon, pA_time = pA
    pI_lat, pI_lon, pI_time = pI
    pB_lat, pB_lon, pB_time = pB

    middle_dist = pI_time - pA_time
    total_dist = pB_time - pA_time
    if total_dist == 0:
        time_ratio = 0
    else:
        time_ratio = middle_dist / total_dist

    lat = pA_lat + (pB_lat - pA_lat) * time_ratio
    lon = pA_lon + (pB_lon - pA_lon) * time_ratio

    lat_diff = lat - pI_lat
    lon_diff = lon - pI_lon
    error = np.sqrt((lat_diff * lat_diff) + (lon_diff * lon_diff))
    return error


def calc_DP(pA, pI, pB):
    """
    It computes the Perpendicular Distance (PD)

    :param pA: initial point
    :param pI: middle point
    :param pB: final point
    :return: shortest distance
    """
    pA_lat, pA_lon, pA_time = pA
    pI_lat, pI_lon, pI_time = pI
    pB_lat, pB_lon, pB_time = pB

    # equation: (yA−yB)x−(xA−xB)y+xAyB−xByA=0.
    A = pA_lon - pB_lon
    B = - (pA_lat - pB_lat)
    C = pA_lat * pB_lon - pB_lat * pA_lon

    if A == 0 and B == 0:
        shortDist = 0
    else:
        shortDist = abs((A * pI_lat + B * pI_lon + C) / np.sqrt(A * A + B * B))

    return shortDist


def calc_AVS(pA, pI, pB):
    """
    It computes the absolute value of speed (AVS)

    :param pA: initial point
    :param pI: middle point
    :param pB: final point
    :return: AVS value
    """
    pA_lat, pA_lon, pA_time = pA
    pI_lat, pI_lon, pI_time = pI
    pB_lat, pB_lon, pB_time = pB

    d1 = np.sqrt((pI_lat - pA_lat) * (pI_lat - pA_lat) + (pI_lon - pA_lon) * (pI_lon - pA_lon))
    d2 = np.sqrt((pB_lat - pI_lat) * (pB_lat - pI_lat) + (pB_lon - pI_lon) * (pB_lon - pI_lon))

    v1 = 0
    v2 = 0
    if (pI_time - pA_time) > 0:
        v1 = d1 / (pI_time - pA_time)
    if (pB_time - pI_time) > 0:
        v2 = d2 / (pB_time - pI_time)
    AVS = abs(v2 - v1)

    return AVS


def calc_TR_SP(trajectory, dim_set, traj_time, epsilon, epsilon2, calc_func, calc_func2):
    """
    It compress the trajectory using the two compression techniques.
    It is a recursive method, dividing the trajectory and compression both parts.

    :param trajectory: a single trajectory or a part of if
    :param dim_set: the attributes in the dict trajectory
    :param traj_time: the array with the time in seconds of each point
    :param epsilon: the threshold for the first compression
    :param epsilon2: the threshold for second compression
    :param calc_func: the measure for the first selected compression
    :param calc_func2: the measure for the second selected compression
    :return: the compressed trajectory (dict)
    """
    new_trajectory = {}
    for dim in dim_set:
        new_trajectory[dim] = np.array([])
    traj_len = len(trajectory['lat'])

    # time in seconds
    dmax, idx, _ = traj_max_dists(trajectory, traj_time, calc_func)
    start_location = (trajectory['lat'][0], trajectory['lon'][0], traj_time[0])
    final_location = (trajectory['lat'][-1], trajectory['lon'][-1], traj_time[-1])
    middle_location = (trajectory['lat'][idx], trajectory['lon'][idx], traj_time[idx])
    d_idx = calc_func2(start_location, middle_location, final_location)

    trajectory['DateTime'] = trajectory['DateTime'].astype(str)

    if (dmax > epsilon) & (d_idx > epsilon2):
        traj1 = {}
        traj2 = {}
        for dim in dim_set:
            traj1[dim] = trajectory[dim][0:idx]
            traj2[dim] = trajectory[dim][idx:]

        # compression of the parts
        recResults1 = traj1
        if len(traj1['lat']) > 2:
            recResults1 = traj_compression(traj1, dim_set, traj_time[0:idx], calc_func, epsilon)

        recResults2 = traj2
        if len(traj2['lat']) > 2:
            recResults2 = traj_compression(traj2, dim_set, traj_time[idx:], calc_func, epsilon)

        for dim in dim_set:
            new_trajectory[dim] = np.append(new_trajectory[dim], recResults1[dim])
            new_trajectory[dim] = np.append(new_trajectory[dim], recResults2[dim])

    else:
        trajectory['DateTime'] = trajectory['DateTime'].astype(str)
        for dim in dim_set:
            new_trajectory[dim] = np.append(new_trajectory[dim], trajectory[dim][0])
            if traj_len > 1:
                new_trajectory[dim] = np.append(new_trajectory[dim], trajectory[dim][-1])

    return new_trajectory


def traj_max_dists(trajectory, traj_time, calc_func):
    """
    It computes the selected measure for all points in between.

    :param trajectory: a single dict trajectory having the keys as each attribute
    :param traj_time: an array with the seconds of each point
    :param calc_func: the measure for the selected compression
    :return: the maximum distance, the index that provide the maximum distance, and the average of distances
    """
    dmax = 0
    idx = 0
    ds = np.array([])
    traj_len = len(trajectory['lat'])
    # start and final points
    start_location = (trajectory['lat'][0], trajectory['lon'][0], traj_time[0])
    final_location = (trajectory['lat'][-1], trajectory['lon'][-1], traj_time[-1])
    for i in range(1, (traj_len - 1)):
        # middle point at index i
        middle = (trajectory['lat'][i], trajectory['lon'][i], traj_time[i])
        # compute the distance
        d = calc_func(start_location, middle, final_location)
        # get distances information
        ds = np.append(ds, d)
        if d > dmax:
            dmax = d
            idx = i

    return dmax, idx, ds.mean()


def traj_compression(trajectory, dim_set, traj_time, calc_func, epsilon):
    """
    It compress the trajectory using the compression technique.
    It is a recursive method, dividing the trajectory and compression both parts.

    :param trajectory: a single trajectory or a part of if
    :param dim_set: the attributes in the dict trajectory
    :param traj_time: the array with the time in seconds of each point
    :param calc_func: the measure for the selected compression
    :param epsilon: the threshold
    :return: the compressed trajectory (dict)
    """
    new_trajectory = {}
    for dim in dim_set:
        new_trajectory[dim] = np.array([])
    traj_len = len(trajectory['lat'])

    # time in seconds
    dmax, idx, _ = traj_max_dists(trajectory, traj_time, calc_func)
    trajectory['DateTime'] = trajectory['DateTime'].astype(str)

    # print(f'\tepsilon: {epsilon}, dmax: {dmax}, index: {idx}, trajlen: {traj_len}')
    if dmax > epsilon:
        traj1 = {}
        traj2 = {}
        for dim in dim_set:
            traj1[dim] = trajectory[dim][0:idx]
            traj2[dim] = trajectory[dim][idx:]

        # compression of the parts
        recResults1 = traj1
        if len(traj1['lat']) > 2:
            recResults1 = traj_compression(traj1, dim_set, traj_time[0:idx], calc_func, epsilon)

        recResults2 = traj2
        if len(traj2['lat']) > 2:
            recResults2 = traj_compression(traj2, dim_set, traj_time[idx:], calc_func, epsilon)

        for dim in dim_set:
            new_trajectory[dim] = np.append(new_trajectory[dim], recResults1[dim])
            new_trajectory[dim] = np.append(new_trajectory[dim], recResults2[dim])

    else:
        trajectory['DateTime'] = trajectory['DateTime'].astype(str)
        for dim in dim_set:
            new_trajectory[dim] = np.append(new_trajectory[dim], trajectory[dim][0])
            if traj_len > 1:
                new_trajectory[dim] = np.append(new_trajectory[dim], trajectory[dim][-1])

    return new_trajectory


def compression(dataset, metric='TR', verbose=True, alpha=1):
    """
    It compress the dataset of trajectories using the selected compression technique.

    :param dataset: dict dataset with trajectories
    :param metric: the compression technique or combination of them (Default: TR).
    :param verbose: if True, it shows the messages (Default: True).
    :param alpha: the predefined factor (Default: 1).
    :return: the compressed dataset, compression rate, and processing time.
    """
    # sys.setrecursionlimit(2200)
    metrics = {'TR': calc_SED,
               'DP': calc_DP,
               'SP': calc_AVS,
               'TR_SP': calc_TR_SP,
               'SP_TR': calc_TR_SP,
               'SP_DP': calc_TR_SP,
               'DP_SP': calc_TR_SP,
               'DP_TR': calc_TR_SP,
               'TR_DP': calc_TR_SP}

    calc_func = metrics[metric]

    mmsis = list(dataset.keys())
    new_dataset = {}
    compression_rate = np.array([])
    processing_time = np.array([])

    dim_set = dataset[mmsis[0]].keys()
    print("Dim set is: ", dim_set)

    if verbose:
        print(f"Compressing with {metric} and factor {alpha}")
    for id_mmsi in range(len(mmsis)):
        new_dataset[mmsis[id_mmsi]] = {}
        if verbose:
            print(f"\tCompressing {id_mmsi} of {len(mmsis)}")
        # trajectory a
        t0 = time.time()
        curr_traj = dataset[mmsis[id_mmsi]]
        # get time in seconds
        traj_time = curr_traj['DateTime'].astype('datetime64[s]')
        traj_time = np.hstack((0, np.diff(traj_time).cumsum().astype('float')))
        traj_time = traj_time / traj_time.max()
        # compress trajectory
        compress_traj = curr_traj
        try:
            if metric in ['TR_SP']:
                max_epsilon, idx, epsilon = traj_max_dists(curr_traj, traj_time, calc_SED)
                max_epsilon2, idx2, epsilon2 = traj_max_dists(curr_traj, traj_time, calc_AVS)
                compress_traj = calc_func(curr_traj, dim_set, traj_time, epsilon * alpha, epsilon2 * alpha, calc_SED,
                                          calc_AVS)
            elif metric in ['SP_TR']:
                max_epsilon, idx, epsilon = traj_max_dists(curr_traj, traj_time, calc_AVS)
                max_epsilon2, idx2, epsilon2 = traj_max_dists(curr_traj, traj_time, calc_SED)
                compress_traj = calc_func(curr_traj, dim_set, traj_time, epsilon * alpha, epsilon2 * alpha, calc_AVS,
                                          calc_SED)
            elif metric in ['SP_DP']:
                max_epsilon, idx, epsilon = traj_max_dists(curr_traj, traj_time, calc_AVS)
                max_epsilon2, idx2, epsilon2 = traj_max_dists(curr_traj, traj_time, calc_DP)
                compress_traj = calc_func(curr_traj, dim_set, traj_time, epsilon * alpha, epsilon2 * alpha, calc_AVS,
                                          calc_DP)
            elif metric in ['TR_DP']:
                max_epsilon, idx, epsilon = traj_max_dists(curr_traj, traj_time, calc_SED)
                max_epsilon2, idx2, epsilon2 = traj_max_dists(curr_traj, traj_time, calc_DP)
                compress_traj = calc_func(curr_traj, dim_set, traj_time, epsilon * alpha, epsilon2 * alpha, calc_SED,
                                          calc_DP)
            elif metric in ['DP_SP']:
                max_epsilon, idx, epsilon = traj_max_dists(curr_traj, traj_time, calc_DP)
                max_epsilon2, idx2, epsilon2 = traj_max_dists(curr_traj, traj_time, calc_AVS)
                compress_traj = calc_func(curr_traj, dim_set, traj_time, epsilon * alpha, epsilon2 * alpha, calc_DP,
                                          calc_AVS)
            elif metric in ['DP_TR']:
                max_epsilon, idx, epsilon = traj_max_dists(curr_traj, traj_time, calc_DP)
                max_epsilon2, idx2, epsilon2 = traj_max_dists(curr_traj, traj_time, calc_SED)
                compress_traj = calc_func(curr_traj, dim_set, traj_time, epsilon * alpha, epsilon2 * alpha, calc_DP,
                                          calc_SED)
            else:
                max_epsilon, idx, epsilon = traj_max_dists(curr_traj, traj_time, calc_func)
                compress_traj = traj_compression(curr_traj, dim_set, traj_time, calc_func, epsilon * alpha)
        except:
            print(
                f"\t\tIt was not possible to compress this trajectory {mmsis[id_mmsi]} of length {len(curr_traj['lat'])}.")

        compress_traj['DateTime'] = compress_traj['DateTime'].astype('datetime64[s]')
        new_dataset[mmsis[id_mmsi]] = compress_traj
        t1 = time.time() - t0
        if verbose:
            print(f"\tlength before: {len(curr_traj['lat'])}, length now: {len(compress_traj['lat'])}, reduction of {1 - len(compress_traj['lat'])/len(curr_traj['lat'])}")
        compression_rate = np.append(compression_rate, 1 - (len(compress_traj['lat']) / len(curr_traj['lat'])))
        processing_time = np.append(processing_time, t1)

    return new_dataset, compression_rate, processing_time

In [4]:
%%time

# Original Compression.
new_dataset = Conversions.pandas_to_dict(dataset)
new_dataset, compression_rate, processing_time = compression(new_dataset, verbose=True)
new_dataset = Conversions.dict_to_pandas(new_dataset)

Dim set is:  dict_keys(['traj_id', 'DateTime', 'lat', 'lon', 'StarkeyTime', 'GMDate', 'GMTime', 'LocDate', 'LocTime', 'RadNum', 'Species', 'UTME', 'UTMN', 'Year', 'Grensunr', 'Grensuns', 'Obswt'])
Compressing with TR and factor 1
	Compressing 0 of 253
	length before: 214, length now: 22, reduction of 0.897196261682243
	Compressing 1 of 253
	length before: 2500, length now: 260, reduction of 0.896
	Compressing 2 of 253
	length before: 1090, length now: 324, reduction of 0.7027522935779816
	Compressing 3 of 253
	length before: 2922, length now: 254, reduction of 0.9130732375085557
	Compressing 4 of 253
	length before: 1776, length now: 130, reduction of 0.9268018018018018
	Compressing 5 of 253
	length before: 2086, length now: 44, reduction of 0.9789069990412272
	Compressing 6 of 253
	length before: 2581, length now: 96, reduction of 0.9628051142967842
	Compressing 7 of 253
	length before: 834, length now: 20, reduction of 0.9760191846522782
	Compressing 8 of 253
	length before: 1625, le

In [8]:
new_dataset

,traj_id,DateTime,lat,lon,StarkeyTime,GMDate,GMTime,LocDate,LocTime,RadNum,Species,UTME,UTMN,Year,Grensunr,Grensuns,Obswt
0,880109D01,1995-04-13 13:40:06,45.239682,-118.533204,229902006.0,21:40:06,19950413.0,19950413.0,13:40:06,409.0,0.0,379662.0,5010734.0,95.0,13:13:00,02:39:00,1.47
1,880109D01,1995-04-16 03:32:14,45.247429,-118.539530,230124734.0,11:32:14,19950416.0,19950416.0,03:32:14,409.0,0.0,379188.0,5011581.0,95.0,13:07:00,02:43:00,1.50
2,880109D01,1995-04-16 04:08:28,45.247117,-118.542579,230126908.0,12:08:28,19950416.0,19950416.0,04:08:28,409.0,0.0,378938.0,5011567.0,95.0,13:07:00,02:43:00,1.34
3,880109D01,1995-04-16 13:35:08,45.245163,-118.527236,230160908.0,21:35:08,19950416.0,19950416.0,13:35:08,409.0,0.0,380150.0,5011327.0,95.0,13:07:00,02:43:00,1.39
4,880109D01,1995-04-16 15:30:54,45.247623,-118.525008,230167854.0,23:30:54,19950416.0,19950416.0,15:30:54,409.0,0.0,380339.0,5011585.0,95.0,13:07:00,02:43:00,1.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12,OSUX93191,1996-07-30 05:17:24,45.228154,-118.586396,270825444.0,13:17:24,19960730.0,19960730.0,05:17:24,390.0,2.0,375477.0,5009522.0,96.0,12:37:00,03:27:00,1.39
13,OSUX93191,1996-07-30 06:46:59,45.231668,-118.586112,270830819.0,14:46:59,19960730.0,19960730.0,06:46:59,390.0,2.0,375501.0,5009911.0,96.0,12:37:00,03:27:00,1.33
14,OSUX93191,1996-08-02 01:48:17,45.204917,-118.587278,271072097.0,09:48:17,19960802.0,19960802.0,01:48:17,390.0,2.0,375359.0,5006953.0,96.0,12:40:00,03:23:00,1.42
15,OSUX93191,1996-08-02 03:33:21,45.204870,-118.590715,271078401.0,11:33:21,19960802.0,19960802.0,03:33:21,390.0,2.0,375062.0,5006939.0,96.0,12:40:00,03:23:00,1.55


In [6]:
%%time

# PTRAIL Compression.
new_dataset1 = Compression.compress_trajectories(dataset, verbose=True)

Compressing with TR and factor 1.
	Compressing 1 of 25
Compressing with TR and factor 1.
	Compressing 1 of 25
Compressing with TR and factor 1.
	Compressing 1 of 25
Compressing with TR and factor 1.
	Compressing 1 of 25
Compressing with TR and factor 1.
	Compressing 1 of 25
Compressing with TR and factor 1.
	Compressing 1 of 25
Compressing with TR and factor 1.
	Compressing 1 of 25
Compressing with TR and factor 1.
Compressing with TR and factor 1.	Compressing 1 of 25Compressing with TR and factor 1.

	Compressing 1 of 25Compressing with TR and factor 1.
	Compressing 1 of 3


	Compressing 1 of 25
	length before: 43, length now: 6, reduction of: 0.0
	Compressing 2 of 3
	length before: 52, length now: 8, reduction of: 0.0
	Compressing 3 of 3	length before: 214, length now: 22, reduction of: 0.0

	Compressing 2 of 25	length before: 384, length now: 34, reduction of: 0.0

	length before: 834, length now: 20, reduction of: 0.0	length before: 52, length now: 6, reduction of: 0.0	Compressing 

In [9]:
new_dataset1

level_0     index        lat         lon  \
traj_id   DateTime                                                        
880109D01 1995-04-13 13:40:06      0.0       0.0  45.239682 -118.533204   
          1995-04-16 03:32:14      3.0       3.0  45.247429 -118.539530   
          1995-04-16 04:08:28      4.0       4.0  45.247117 -118.542579   
          1995-04-16 13:35:08     14.0      14.0  45.245163 -118.527236   
          1995-04-16 15:30:54     15.0      15.0  45.247623 -118.525008   
...                                ...       ...        ...         ...   
OSUX93191 1996-07-30 05:17:24  11201.0  286983.0  45.228154 -118.586396   
          1996-07-30 06:46:59  11202.0  286984.0  45.231668 -118.586112   
          1996-08-02 01:48:17  11226.0  287008.0  45.204917 -118.587278   
          1996-08-02 03:33:21  11227.0  287009.0  45.204870 -118.590715   
          1996-08-15 14:25:58  11353.0  287135.0  45.222562 -118.541151   

                               StarkeyTime    GMDate      GMTime     LocDate  \
traj_id   DateTime                                                             
880109D01 1995-04-13 13:40:06  229902006.0  21:40:06  19950413.0  19950413.0   
          1995-04-16 03:32:14  230124734.0  11:32:14  19950416.0  19950416.0   
          1995-04-16 04:08:28  230126908.0  12:08:28  19950416.0  19950416.0   
          1995-04-16 13:35:08  230160908.0  21:35:08  19950416.0  19950416.0   
          1995-04-16 15:30:54  230167854.0  23:30:54  19950416.0  19950416.0   
...                                    ...       ...         ...         ...   
OSUX93191 1996-07-30 05:17:24  270825444.0  13:17:24  19960730.0  19960730.0   
          1996-07-30 06:46:59  270830819.0  14:46:59  19960730.0  19960730.0   
          1996-08-02 01:48:17  271072097.0  09:48:17  19960802.0  19960802.0   
          1996-08-02 03:33:21  271078401.0  11:33:21  19960802.0  19960802.0   
          1996-08-15 14:25:58  272240758.0  22:25:58  19960815.0  19960815.0   

                                LocTime  RadNum  Species      UTME       UTMN  \
traj_id   DateTime                                                              
880109D01 1995-04-13 13:40:06  13:40:06   409.0      0.0  379662.0  5010734.0   
          1995-04-16 03:32:14  03:32:14   409.0      0.0  379188.0  5011581.0   
          1995-04-16 04:08:28  04:08:28   409.0      0.0  378938.0  5011567.0   
          1995-04-16 13:35:08  13:35:08   409.0      0.0  380150.0  5011327.0   
          1995-04-16 15:30:54  15:30:54   409.0      0.0  380339.0  5011585.0   
...                                 ...     ...      ...       ...        ...   
OSUX93191 1996-07-30 05:17:24  05:17:24   390.0      2.0  375477.0  5009522.0   
          1996-07-30 06:46:59  06:46:59   390.0      2.0  375501.0  5009911.0   
          1996-08-02 01:48:17  01:48:17   390.0      2.0  375359.0  5006953.0   
          1996-08-02 03:33:21  03:33:21   390.0      2.0  375062.0  5006939.0   
          1996-08-15 14:25:58  14:25:58   390.0      2.0  378995.0  5008820.0   

                               Year  Grensunr  Grensuns  Obswt  
traj_id   DateTime                                              
880109D01 1995-04-13 13:40:06  95.0  13:13:00  02:39:00   1.47  
          1995-04-16 03:32:14  95.0  13:07:00  02:43:00   1.50  
          1995-04-16 04:08:28  95.0  13:07:00  02:43:00   1.34  
          1995-04-16 13:35:08  95.0  13:07:00  02:43:00   1.39  
          1995-04-16 15:30:54  95.0  13:07:00  02:43:00   1.55  
...                             ...       ...       ...    ...  
OSUX93191 1996-07-30 05:17:24  96.0  12:37:00  03:27:00   1.39  
          1996-07-30 06:46:59  96.0  12:37:00  03:27:00   1.33  
          1996-08-02 01:48:17  96.0  12:40:00  03:23:00   1.42  
          1996-08-02 03:33:21  96.0  12:40:00  03:23:00   1.55  
          1996-08-15 14:25:58  96.0  12:56:00  03:04:00   1.66  

[21649 rows x 17 columns]